In [10]:
import os
import numpy as np
import pandas as pd
import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras_adabound import AdaBound

from sklearn.model_selection import train_test_split

def build_model(size, num_classes):
    inputs = Input((size, size, 3))
    backbone = MobileNetV2(input_tensor = inputs, include_top=False, weights = "imagenet")
    backbone.trainable = True
    x = backbone.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(num_classes, activation="softmax")(x)
    
    model = tf.keras.Model(inputs, x)
    return model

def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size, size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

def parse_data(x, y):
    x = x.decode()
    
    num_class = 120
    size = 224
    
    image = read_image(x, size)
    label = [0] * num_class
    label[y] = 1
    label = np.array(label)
    label = label.astype(np.int32)
    
    return image, label

def tf_parse(x, y):
    x, y = tf.numpy_function(parse_data, [x, y], [tf.float32, tf.int32])
    x.set_shape((224, 224, 3))
    y.set_shape((120))
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

if __name__ == "__main__":
    path = "dog-breed-identification/"
    train_path = os.path.join(path, "train/*")
    test_path = os.path.join(path, "test/*")
    labels_path = os.path.join(path, "labels.csv")
    
    labels_df = pd.read_csv(labels_path)
    breed = labels_df["breed"].unique()
    print("Number of breeds: ", len(breed))
          
    breed2id = {name: i for i, name in enumerate(breed)}
        
    ids = glob(train_path)
    labels = []
    
    for image_id in ids:
        image_id = image_id.split("\\")[-1].split(".")[0]
        breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
        breed_idx = breed2id[breed_name]
        labels.append(breed_idx)
        
    ids = ids[:1000]
    labels = labels[:1000]
        
    ##Splitting data set
    
    train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42) ##test size 20%
    train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)
    
    ##parameters
    size = 224
    num_classes = 120
    lr = 1e-4
    batch = 32
    epochs = 20
    
    ## Building the model
    
    model = build_model(size, num_classes)
    opt = tf.keras.optimizers.Adam(lr=lr)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])
    

    ## Dataset
    
    train_dataset = tf_dataset(train_x, train_y, batch = batch)
    valid_dataset = tf_dataset(valid_x, valid_y, batch = batch)
    
    ## Training
    
    callbacks = [
        ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
        ReduceLROnPlateau(factor = 0.2, patience = 5, min_lr = 1e-6)
    ]
    
    train_steps = (len(train_x)//batch) + 1
    valid_steps = (len(valid_x)//batch) + 1
    model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        validation_data=valid_dataset,
        epochs=epochs,
        callbacks=callbacks)
        

Number of breeds:  120
Epoch 1/20
26/26 [==============================] - ETA: 0s - loss: 4.8075 - acc: 0.0445
Epoch 00001: val_loss improved from inf to 4.53964, saving model to model.h5
26/26 [==============================] - 168s 6s/step - loss: 4.8075 - acc: 0.0445 - val_loss: 4.5396 - val_acc: 0.0500
Epoch 2/20
26/26 [==============================] - ETA: 0s - loss: 3.3470 - acc: 0.4820
Epoch 00002: val_loss improved from 4.53964 to 4.19518, saving model to model.h5
26/26 [==============================] - 169s 7s/step - loss: 3.3470 - acc: 0.4820 - val_loss: 4.1952 - val_acc: 0.1300
Epoch 3/20
26/26 [==============================] - ETA: 0s - loss: 2.0943 - acc: 0.8257
Epoch 00003: val_loss improved from 4.19518 to 3.75214, saving model to model.h5
26/26 [==============================] - 187s 7s/step - loss: 2.0943 - acc: 0.8257 - val_loss: 3.7521 - val_acc: 0.2000
Epoch 4/20
26/26 [==============================] - ETA: 0s - loss: 1.0566 - acc: 0.9627
Epoch 00004: val_loss 

In [4]:
pip install keras-adabound

  Created wheel for keras-adabound: filename=keras_adabound-0.6.0-py3-none-any.whl size=6608 sha256=6fe272cb390a953dbbc3ec91a219d4eb2a6521c00890202962c7418b84bafa66
  Stored in directory: c:\users\ericg\appdata\local\pip\cache\wheels\bc\8a\34\f7fd8b562417782e64c61fd17829dc5a153a46ec58065aa570
Successfully built keras-adabound

